# llama3-hackathon

In [ ]:
! pip install llama-index
! pip install pydantic
! pip install groq
! pip install llama-index-llms-groq
! pip install llama-index-llms-together
! pip install substrate
! pip install yfinance
! pip install pandas

In [7]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq

# from google.colab import userdata

from llama_index.core.llms import ChatMessage, MessageRole

import yfinance as yf
from typing import Annotated, Callable, Any, Optional
from pandas import DataFrame
from functools import wraps

from llama_index.llms.together import TogetherLLM

In [5]:
# Utility function for saving output
def save_output(
    data: DataFrame, description: str, save_path: Optional[str] = None
) -> None:
    if save_path:
        data.to_csv(save_path)
        print(f"{description} saved to {save_path}")


def get_stock_data(
    symbol: str, start_date: str, end_date: str, save_path: Optional[str] = None
) -> str:
    # Fetch stock data from Yahoo Finance
    ticker = yf.Ticker(symbol)
    stock_data = ticker.history(start=start_date, end=end_date).describe()
    save_output(stock_data, f"Stock data for {ticker.ticker}", save_path)
    return stock_data.to_string()


"""
def get_stock_info(symbol: str) -> str:
    # Fetch stock information from Yahoo Finance
    ticker = yf.Ticker(symbol)
    info = ticker.info
    return str(info)

def get_company_info(symbol: str, save_path: Optional[str] = None) -> str:
    # Fetch company information from Yahoo Finance
    ticker = yf.Ticker(symbol)
    info = ticker.info
    company_info = {
        "Company Name": info.get("shortName", "N/A"),
        "Industry": info.get("industry", "N/A"),
        "Sector": info.get("sector", "N/A"),
        "Country": info.get("country", "N/A"),
        "Website": info.get("website", "N/A"),
    }
    company_info_df = DataFrame([company_info])
    if save_path:
        company_info_df.to_csv(save_path)
        print(f"Company info for {ticker.ticker} saved to {save_path}")
    return company_info_df.to_string()

def get_stock_dividends(symbol: str, save_path: Optional[str] = None) -> str:
    # Fetch stock dividends from Yahoo Finance
    ticker = yf.Ticker(symbol)
    dividends = ticker.dividends
    if save_path:
        dividends.to_csv(save_path)
        print(f"Dividends for {ticker.ticker} saved to {save_path}")
    return dividends.to_string()

def get_income_stmt(symbol: str) -> str:
    # Fetch income statement from Yahoo Finance
    ticker = yf.Ticker(symbol)
    income_stmt = ticker.financials
    return income_stmt.to_string()

def get_balance_sheet(symbol: str) -> str:
    # Fetch balance sheet from Yahoo Finance
    ticker = yf.Ticker(symbol)
    balance_sheet = ticker.balance_sheet
    return balance_sheet.to_string()

def get_cash_flow(symbol: str) -> str:
    # Fetch cash flow from Yahoo Finance
    ticker = yf.Ticker(symbol)
    cash_flow = ticker.cashflow
    return cash_flow.to_string()
"""


def get_analyst_recommendations(symbol: str) -> str:
    # Fetch analyst recommendations from Yahoo Finance
    ticker = yf.Ticker(symbol)
    recommendations = ticker.recommendations
    if recommendations.empty:
        return "No recommendations available"

    recommendation_counts = recommendations[
        ["strongBuy", "buy", "hold", "sell", "strongSell"]
    ].sum()
    most_common_recommendation = recommendation_counts.idxmax()
    count = recommendation_counts.max()

    return f"Most common recommendation: {most_common_recommendation} ({count} votes)"

In [6]:
# Set up the Groq class with the required model and API key
llm = Groq(
    model="llama3-8b-8192",
    api_key="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",
)

In [8]:
llm3 = TogetherLLM(
    model="meta-llama/Meta-Llama-3-8B-Instruct-Lite",
    api_key="d0ca8bc44aac868cafe8e100a0e3698183000c8562dc76a286cb96f1b9d8cdc6",
)

In [12]:
from llama_index.core import PromptTemplate


def get_balance_sheet(
    symbol: str,
) -> str:
    # Fetch balance sheet from Yahoo Finance
    ticker = yf.Ticker(symbol)
    balance_sheet = ticker.balance_sheet

    # Prompt template for summarizing financial data
    template_financial_summary = """You are a financial analyst. Your task is to provide a brief summary of the following financial data concisley in less than 50 words.\n\n{balance_sheet}"""

    prompt_balance_sheet = PromptTemplate(template=template_financial_summary)

    # Use Llama Index with chosen LLM model
    result_summary = llm.complete(
        prompt_balance_sheet.format(balance_sheet=balance_sheet)
    )

    return result_summary.text


def get_stock_info(
    symbol: str,
) -> str:
    # Fetch stock information from Yahoo Finance
    ticker = yf.Ticker(symbol)
    info = ticker.info

    stock_info = str(info)

    # Prompt template for summarizing financial data
    template_stock_prices = """You are a financial analyst. Your task is to provide a brief summary of insights based based on stock information in maximum 3 lines.\n\n{stock_info}"""

    prompt_stock_proces = PromptTemplate(template=template_stock_prices)

    # Use Llama Index with chosen LLM model
    result_summary = llm.complete(prompt_stock_proces.format(stock_info=stock_info))

    return result_summary.text


def get_company_info(
    symbol: str,
    save_path: Optional[str] = None,
) -> str:
    # Fetch company information from Yahoo Finance
    ticker = yf.Ticker(symbol)
    info = ticker.info
    company_info = {
        "Company Name": info.get("shortName", "N/A"),
        "Industry": info.get("industry", "N/A"),
        "Sector": info.get("sector", "N/A"),
        "Country": info.get("country", "N/A"),
        "Website": info.get("website", "N/A"),
    }
    company_info_df = DataFrame([company_info])
    if save_path:
        company_info_df.to_csv(save_path)
        print(f"Company info for {ticker.ticker} saved to {save_path}")

    company_info = str(company_info_df)

    # Prompt template for summarizing financial data
    template_company_info = """You are a financial analyst. Your task is to summarize the key insights you can derive based on content.\n\n{company_info}"""

    prompt_company_info = PromptTemplate(template=template_company_info)

    # Use Llama Index with chosen LLM model
    result_summary = llm.complete(prompt_company_info.format(company_info=company_info))

    return result_summary.text


def get_income_stmt(
    symbol: str,
) -> str:
    # Fetch income statement from Yahoo Finance
    ticker = yf.Ticker(symbol)
    income_stmt = ticker.financials

    income_stmt = income_stmt.to_string()

    # Prompt template for summarizing financial data
    template_income_stmt = """You are a financial analyst. Your task is to summarize the key insights you can derive based on the following content in 3 lines.\n\n{income_stmt}"""
    prompt_income_stmt = PromptTemplate(template=template_income_stmt)
    # Use Llama Index with chosen LLM model
    result_summary = llm.complete(prompt_income_stmt.format(income_stmt=income_stmt))
    return result_summary.text


def get_cash_flow(
    symbol: str,
) -> str:
    # Fetch cash flow from Yahoo Finance
    ticker = yf.Ticker(symbol)
    cash_flows = ticker.cashflow
    cash_flows = cash_flows.to_string()

    # Prompt template for summarizing financial data
    template_cash_flows_template = """You are a financial analyst. Your task is to summarize the key insights you can derive based on the following content.\n\n{cash_flows}"""

    prompt_cash_flows = PromptTemplate(template=template_cash_flows_template)

    # Use Llama Index with chosen LLM model
    result_summary = llm.complete(prompt_cash_flows.format(cash_flows=cash_flows))

    return result_summary.text


def get_stock_dividends(
    symbol: str,
    save_path: Optional[str] = None,
) -> str:
    # Fetch stock dividends from Yahoo Finance
    ticker = yf.Ticker(symbol)
    dividends = ticker.dividends
    if save_path:
        dividends.to_csv(save_path)
        print(f"Dividends for {ticker.ticker} saved to {save_path}")

    dividends_info = dividends.to_string()

    # Prompt template for summarizing financial data
    template_dividends_info = """You are a financial analyst. Your task is to summarize the key trend you can derive based on following content in concise manner.\n\n{dividends_info}"""

    prompt_dividends_info = PromptTemplate(template=template_dividends_info)

    # Use Llama Index with chosen LLM model
    result_summary = llm.complete(
        prompt_dividends_info.format(dividends_info=dividends_info)
    )

    return result_summary.text


print(type(get_stock_dividends("MSFT")))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [15]:
# Define tools for each function
get_stock_data_tool = FunctionTool.from_defaults(fn=get_stock_data)
get_stock_info_tool = FunctionTool.from_defaults(fn=get_stock_info)
get_company_info_tool = FunctionTool.from_defaults(fn=get_company_info)
get_stock_dividends_tool = FunctionTool.from_defaults(fn=get_stock_dividends)
get_income_stmt_tool = FunctionTool.from_defaults(fn=get_income_stmt)
get_balance_sheet_tool = FunctionTool.from_defaults(fn=get_balance_sheet)
get_cash_flow_tool = FunctionTool.from_defaults(fn=get_cash_flow)
get_analyst_recommendations_tool = FunctionTool.from_defaults(
    fn=get_analyst_recommendations
)


GPT_FINANCIAL_ANALYST_SYS_STR = (
    react_system_header_str
) = """
You are a Financial Analyst. Assist users with financial analysis, investment advice, budgeting, market analysis, and financial planning.

1) Financial Analysis: Analyze financial statements, ratios, performance.
2) Investment Analysis: Provide stock/bond insights, risk/return analysis, valuations, portfolio advice.
3) Budgeting and Forecasting: Assist with budgets, forecasting, variance analysis.
4) Market Analysis: Market research, economic trends, competitive analysis.
5) Financial Planning: Personal financial planning, debt management, goal setting.

Guidelines:
- Stick to financial analysis.
- Limit tool output to 800 tokens.
- Use appropriate financial terminology.
- Base advice on sound principles and current market conditions.

Before response, I want you to think through is that the response generated is correct and elaborate on your reasoning before you give the final answer.
"""
# - Provide clear, concise, and comprehensive responses.


prefix_msgs = [
    ChatMessage(role=MessageRole.SYSTEM, content=GPT_FINANCIAL_ANALYST_SYS_STR)
]

aa = ReActAgent.from_tools(
    [
        get_stock_data_tool,
        get_stock_info_tool,
        get_company_info_tool,
        get_stock_dividends_tool,
        get_income_stmt_tool,
        get_balance_sheet_tool,
        get_cash_flow_tool,
        get_analyst_recommendations_tool,
    ],
    llm=llm3,
    prefix_messages=prefix_msgs,
    token_counting=True,
    verbose=True,
    max_iterations=20,
)

In [16]:
# @markdown ---
# @markdown ### Enter a Financial Query:
query = "I have 200 intel shared bought for $40 avg.; what should i do with that? Sell, buy more? else what"  # @param {type:"string"}
aa.reset()
response = aa.chat(query)
print(response)

> Running step ad8f89ed-f461-4ebf-b8be-a41bb6ec1038. Step input: I have 200 intel shared bought for $40 avg.; what should i do with that? Sell, buy more? else what
Thought: The user is seeking advice on what to do with their Intel stock, which they purchased at an average price of $40. I need to use a tool to help me answer the question.
Action: get_stock_info
Action Input: {'symbol': 'INTC'}
Observation: Here is a brief summary of insights based on the stock information:

* Intel Corporation (INTC) is a technology company that designs, develops, manufactures, and sells computing and related products and services worldwide.
* The company's stock price has been relatively stable, with a current price of $32.98 and a 52-week range of $29.73 to $51.28.
* Intel's financial performance has been mixed, with a trailing EPS of $0.97 and a forward EPS of $1.92, but a declining revenue growth rate of 0.086%.
* The company's valuation metrics are relatively high, with a price-to-earnings ratio of

In [ ]:
api_key = "apik_tMzRgPeGtRkF9fSWW8vSr0baSgz6wbRU"
if api_key is None:
    raise EnvironmentError("No SUBSTRATE_API_KEY set")

from substrate import Substrate, ComputeText, sb

substrate = Substrate(api_key=api_key, timeout=60 * 5)

story = ComputeText(prompt="tell me a story", model="Llama3Instruct8B")
summary = ComputeText(
    prompt=sb.concat("Summarize this story: ", story.future.text),
    model="Llama3Instruct8B",
)
response = substrate.run(story)

story_out = response.get(story)
print(story_out.text)

In [19]:
!


      👋 Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  ^C
2024-07-21 01:14:14.218 
